In [3]:
batch_size = 64
input_size = 784  
num_classes = 1   
num_epochs = 10  
learning_rate = 0.0001  
momentum = 0.0 


import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torchvision import datasets, transforms, utils
from torch.utils.data.sampler import SubsetRandomSampler
from torch.nn import functional as F
import matplotlib.pyplot as plt


train_data = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)
                   
test_data = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

subset_indices = ((train_data.train_labels == 0) + (train_data.train_labels == 1)).nonzero().view(-1)

train_loader = torch.utils.data.DataLoader(dataset=train_data, 
                                           batch_size=batch_size, 
                                           shuffle=False,
                                           sampler=SubsetRandomSampler(subset_indices))

subset_indices = ((test_data.test_labels == 0) + (test_data.test_labels == 1)).nonzero().view(-1)

test_loader = torch.utils.data.DataLoader(dataset=test_data, 
                                          batch_size=batch_size,
                                          shuffle=False,
                                          sampler=SubsetRandomSampler(subset_indices))



class SVM_Loss(nn.modules.Module):    
    def __init__(self):
        super(SVM_Loss,self).__init__()
    def forward(self, outputs, labels):
         return torch.sum(torch.clamp(1 - outputs.t()*labels, min=0))/batch_size


svm_model = nn.Linear(input_size,num_classes)

svm_loss_criteria = SVM_Loss()


svm_optimizer = torch.optim.SGD(svm_model.parameters(), lr=learning_rate, momentum=momentum)

total_step = len(train_loader)
for epoch in range(num_epochs):
    avg_loss_epoch = 0
    batch_loss = 0
    total_batches = 0
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, 28*28)                      
        labels = Variable(2*(labels.float()-0.5))
                
        # Forward pass        
        outputs = svm_model(images)           
        loss_svm = svm_loss_criteria(outputs, labels)    
        
       
        # Backward and optimize
        svm_optimizer.zero_grad()
        loss_svm.backward()
        svm_optimizer.step()    
        
        
        total_batches += 1     
        batch_loss += loss_svm.item()

    avg_loss_epoch = batch_loss/total_batches
    print ('Epoch [{}/{}], Averge Loss:for epoch[{}, {:.4f}]' 
                   .format(epoch+1, num_epochs, epoch+1, avg_loss_epoch ))
        

        # Test the SVM Model
correct = 0.
total = 0.
for images, labels in test_loader:
    images = images.reshape(-1, 28*28)
    
    outputs = svm_model(images)    
    predicted = outputs.data >= 0
    total += labels.size(0) 
    correct += (predicted.view(-1).long() == labels).sum()    
 
print('Accuracy of the model on the test images: %f %%' % (100 * (correct.float() / total)))
print("the learning rate is ", learning_rate)
print( "the momentum is", momentum)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:52: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Epoch [1/10], Averge Loss:for epoch[1, 0.8875]
Epoch [2/10], Averge Loss:for epoch[2, 0.6298]
Epoch [3/10], Averge Loss:for epoch[3, 0.4318]
Epoch [4/10], Averge Loss:for epoch[4, 0.2974]
Epoch [5/10], Averge Loss:for epoch[5, 0.1835]
Epoch [6/10], Averge Loss:for epoch[6, 0.1208]
Epoch [7/10], Averge Loss:for epoch[7, 0.0919]
Epoch [8/10], Averge Loss:for epoch[8, 0.0758]
Epoch [9/10], Averge Loss:for epoch[9, 0.0654]
Epoch [10/10], Averge Loss:for epoch[10, 0.0583]
Accuracy of the model on the test images: 99.905434 %
the learning rate is  0.0001
the momentum is 0.0
